# Classificação de Discurso de Ódio 
-------------------------------------
## Abordagem: Bert com pos_weight na BCEWithLogitsLoss

-------------------------------------

## Importações e Definições

In [1]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import torch

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

from utils import preprocess_text, format_lime_output, print_multilabel_metrics

[nltk_data] Downloading package stopwords to /home/penido/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
target = ['aggressive', 'hate', 'ageism', 'aporophobia', 'body_shame', 'capacitism', 'lgbtphobia', 'political', 'racism', 'religious_intolerance', 'misogyny', 'xenophobia', 'other']
features = 'text'

In [3]:
# Configurações para o pré-processamento
config = {
    "lowercase": True,  # se BERT uncased
    "remove_accents": False,
    "remove_punctuation": False,
    "remove_numbers": False,
    "remove_urls": True,
    "remove_mentions_hashtags": True,
    "expand_abbreviations": True,
    "expand_contractions": False,
    "normalize_laughter": True,
    "remove_emojis": False,  # ou substitua por <EMOJI>
    "remove_stopwords": False,
    "lemmatize": False,
    "stemming": False,
    "pos_filter": False,
    "min_token_length": 2,
    "negation_scope": False,
    "replace_swears": False,  # manter palavrões é útil para detecção de discurso de ódio
    "split_hashtags": True,
    "merge_mwes": True,  # só se o seu dicionário for bom
    "replace_named_entities": False
}

le = LabelEncoder()

--------------------------------------------
## Prepara o Conjunto de Dados

In [4]:
def undersample_majority_class(df, label_col='label', majority_class=0, desired_ratio=0.3, random_state=42):
    # Separa a classe majoritária (classe 0)
    df_majority = df[df[label_col] == majority_class]

    # Separa todas as outras classes
    df_others = df[df[label_col] != majority_class]

    # Número total de amostras desejado após o balanceamento
    total_desired = len(df_others) / (1 - desired_ratio)

    # Quantidade desejada de amostras da classe majoritária
    n_majority_desired = int(total_desired * desired_ratio)

    # Undersample da classe majoritária
    df_majority_downsampled = df_majority.sample(n=n_majority_desired, random_state=random_state)

    # Junta os dados
    df_balanced = pd.concat([df_majority_downsampled, df_others])

    # Embaralha (opcional mas recomendado)
    df_balanced = df_balanced.sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_balanced


In [5]:
# 1. Carrega o dataset TuPyE multilabel
df = load_dataset("Silly-Machine/TuPyE-Dataset", name="multilabel")

train_df = df['train'].to_pandas()
train_df["label_comb"] = train_df[target].astype(str).agg("".join, axis=1)
train_df["class_id"] = le.fit_transform(train_df["label_comb"])

train_df = undersample_majority_class(train_df, label_col='class_id', majority_class=0, desired_ratio=0.5)

test_df = df['test'].to_pandas()

X_train_raw = train_df[features]
y_train = train_df[target].values

X_test_raw = test_df[features]
y_test = test_df[target].values

### Aplica Pré-Processamento

In [6]:
X_train = X_train_raw.apply(lambda x: preprocess_text(x, config))
X_test = X_test_raw.apply(lambda x: preprocess_text(x, config))

--------------------------------------------
## Treinamento do Modelo

In [7]:
# 2. Tokenizador BERT em português
model_name = "neuralmind/bert-base-portuguese-cased"
# model_name= "neuralmind/bert-large-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [8]:
# 3. Crie seu dataset personalizado
class MultilabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_dataset = MultilabelDataset(X_train.tolist(), y_train, tokenizer)
test_dataset = MultilabelDataset(X_test.tolist(), y_test, tokenizer)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
y_train_tensor = torch.tensor(y_train)  # ou use diretamente o tensor

# Contagem de positivos e negativos por classe
pos_counts = y_train_tensor.sum(dim=0)
neg_counts = y_train_tensor.shape[0] - pos_counts

# Peso positivo = (n negativos) / (n positivos)
pos_weight = neg_counts / (pos_counts + 1e-5)  # evita divisão por zero
pos_weight = torch.min(pos_weight, torch.tensor(10.0))
pos_weight = pos_weight.to(device)

In [12]:
pos_weight

tensor([ 1.2607,  3.8377, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000, 10.0000,
        10.0000, 10.0000, 10.0000, 10.0000,  4.6498], device='cuda:0')

In [13]:
import torch.nn as nn

class MultilabelTrainer(Trainer):
    def __init__(self, *args, pos_weight=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_weight = pos_weight
        self.loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)

    def compute_loss(self, model, inputs,num_items_in_batch, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = self.loss_fn(logits, labels.float())  # BCE precisa de float
        return (loss, outputs) if return_outputs else loss

In [14]:
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=len(target), problem_type="multi_label_classification")
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [15]:
training_args = TrainingArguments(
    output_dir='./results',       # Ainda obrigatório, mas o Trainer não vai salvar nada
    per_device_train_batch_size=10,
    per_device_eval_batch_size=8,
    num_train_epochs=4,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="no",           # <-- desativa o salvamento de modelos/checkpoints
)

In [16]:
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
# )

trainer = MultilabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    pos_weight=pos_weight 
)

In [17]:
trainer.train()

Step,Training Loss
50,0.650500
100,0.555800
150,0.583900
200,0.485300
250,0.486300
300,0.472600
350,0.440000
400,0.418300
450,0.435800
500,0.435100


TrainOutput(global_step=8132, training_loss=0.2450932721111262, metrics={'train_runtime': 4695.6655, 'train_samples_per_second': 17.316, 'train_steps_per_second': 1.732, 'total_flos': 5349049777741824.0, 'train_loss': 0.2450932721111262, 'epoch': 4.0})

-----------------------------------
## Avaliação no conjunto de Teste

In [18]:
def predict(text):
    model.eval()  # Garante que o modelo está em modo de avaliação
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Tokenização e envio dos inputs para o mesmo device do modelo
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()[0]  # move o resultado de volta para CPU para conversão

    return dict(zip(target, probs))

In [19]:
# Exemplo:
print(predict("Esse cara é um idiota inútil"))

{'aggressive': 0.990462, 'hate': 0.03183589, 'ageism': 0.0008927432, 'aporophobia': 0.0016496172, 'body_shame': 0.0012331516, 'capacitism': 0.0031170864, 'lgbtphobia': 0.002713505, 'political': 0.008915195, 'racism': 0.002239556, 'religious_intolerance': 0.0026345241, 'misogyny': 0.0034058397, 'xenophobia': 0.0015203615, 'other': 0.04311796}


In [20]:
# Exemplo:
print(predict("bom dia"))

{'aggressive': 0.008286432, 'hate': 0.0030249502, 'ageism': 0.0013711029, 'aporophobia': 0.00087756943, 'body_shame': 0.002387941, 'capacitism': 0.0009954101, 'lgbtphobia': 0.0029321609, 'political': 0.0026831757, 'racism': 0.004378396, 'religious_intolerance': 0.0014177023, 'misogyny': 0.003859935, 'xenophobia': 0.004413594, 'other': 0.011459826}


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [22]:
from torch.utils.data import DataLoader

test_loader = DataLoader(test_dataset, batch_size=16)

all_probs = []
all_labels = []

In [23]:
from tqdm import tqdm

# 2. Inferência por batches no test set
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Predicting"):
        inputs = {k: v.to(device) for k, v in batch.items() if k != "labels"}
        labels = batch["labels"].cpu().numpy()
        outputs = model(**inputs)
        probs = torch.sigmoid(outputs.logits).cpu().numpy()

        all_probs.append(probs)
        all_labels.append(labels)

# 3. Concatenar tudo
all_probs = np.concatenate(all_probs, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

Predicting: 100%|██████████| 546/546 [03:25<00:00,  2.66it/s]


In [24]:
all_probs[0].shape

(13,)

In [25]:
preds = (all_probs >= 0.5).astype(int)

In [26]:
print_multilabel_metrics(all_labels, preds)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.6266
✔️ F1 Score (Macro):     0.4656
✔️ F1 Score (Weighted):  0.6308
⚠️ Hamming Loss:         0.0402
✅ Subset Accuracy:      0.7005


In [27]:
# Considerando só agressive e hate
print_multilabel_metrics(all_labels[:, :2], preds[:, :2])


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.6718
✔️ F1 Score (Macro):     0.6585
✔️ F1 Score (Weighted):  0.6719
⚠️ Hamming Loss:         0.1419
✅ Subset Accuracy:      0.7845


In [28]:
from sklearn.metrics import f1_score

def otimizar_thresholds_weighted(y_true, y_pred_proba, thresholds=np.arange(0.0, 1.01, 0.01)):
    """
    Otimiza o threshold para cada label individualmente maximizando o F1-score (weighted).

    Parâmetros:
        y_true: ndarray binário de shape (n_amostras, n_labels)
        y_pred_proba: ndarray de probabilidades (entre 0 e 1), mesma shape
        thresholds: array de thresholds a serem testados

    Retorna:
        thresholds_otimos: lista com melhor threshold para cada label
        f1_scores: lista com F1-score (weighted) correspondente ao melhor threshold
    """
    n_labels = y_true.shape[1]
    thresholds_otimos = []
    f1_scores = []

    for i in tqdm(range(n_labels), desc="Otimizando thresholds (F1 weighted)"):
        f1_max = -1
        best_thresh = 0.5
        for t in thresholds:
            y_pred_bin = (y_pred_proba[:, i] >= t).astype(int)
            f1 = f1_score(y_true[:, i], y_pred_bin, average='weighted', zero_division=0)
            if f1 > f1_max:
                f1_max = f1
                best_thresh = t
        thresholds_otimos.append(best_thresh)
        f1_scores.append(f1_max)

    return thresholds_otimos, f1_scores


In [29]:
from sklearn.metrics import accuracy_score

def otimizar_thresholds_acuracia(y_true, y_pred_proba, thresholds=np.arange(0.0, 1.01, 0.01)):
    """
    Encontra o melhor threshold para cada label maximizando a acurácia.

    Parâmetros:
        y_true: ndarray binário de shape (n_amostras, n_labels)
        y_pred_proba: ndarray de probabilidades (n_amostras, n_labels)
        thresholds: array de valores de corte a testar

    Retorna:
        thresholds_otimos: lista com melhor threshold para cada label
        acc_scores: lista com acurácia correspondente a cada threshold ótimo
    """
    n_labels = y_true.shape[1]
    thresholds_otimos = []
    acc_scores = []

    for i in tqdm(range(n_labels), desc="Otimizando thresholds (Acurácia)"):
        acc_max = -1
        best_thresh = 0.5
        for t in thresholds:
            y_pred_bin = (y_pred_proba[:, i] >= t).astype(int)
            acc = accuracy_score(y_true[:, i], y_pred_bin)
            if acc > acc_max:
                acc_max = acc
                best_thresh = t
        thresholds_otimos.append(best_thresh)
        acc_scores.append(acc_max)

    return thresholds_otimos, acc_scores


In [30]:
thresholds_otimos, f1s = otimizar_thresholds_acuracia(all_labels, all_probs)

for i, (t, f) in enumerate(zip(thresholds_otimos, f1s)):
    print(f"Label {i}: Threshold ótimo = {t:.2f}, F1 weighted = {f:.4f}")

Otimizando thresholds (Acurácia): 100%|██████████| 13/13 [00:08<00:00,  1.48it/s]

Label 0: Threshold ótimo = 0.82, F1 weighted = 0.8505
Label 1: Threshold ótimo = 0.90, F1 weighted = 0.9132
Label 2: Threshold ótimo = 0.93, F1 weighted = 0.9986
Label 3: Threshold ótimo = 0.70, F1 weighted = 0.9987
Label 4: Threshold ótimo = 0.93, F1 weighted = 0.9956
Label 5: Threshold ótimo = 0.91, F1 weighted = 0.9987
Label 6: Threshold ótimo = 0.81, F1 weighted = 0.9916
Label 7: Threshold ótimo = 0.95, F1 weighted = 0.9807
Label 8: Threshold ótimo = 1.00, F1 weighted = 0.9936
Label 9: Threshold ótimo = 0.99, F1 weighted = 0.9981
Label 10: Threshold ótimo = 0.98, F1 weighted = 0.9745
Label 11: Threshold ótimo = 0.99, F1 weighted = 0.9904
Label 12: Threshold ótimo = 0.97, F1 weighted = 0.9171


In [31]:
preds = np.zeros_like(all_probs, dtype=int)

for i, threshold in enumerate(thresholds_otimos):
    preds[:, i] = (all_probs[:, i] >= threshold).astype(int)

In [32]:
print_multilabel_metrics(all_labels, preds)


📊 Avaliação Multilabel
✔️ F1 Score (Micro):     0.6213
✔️ F1 Score (Macro):     0.3875
✔️ F1 Score (Weighted):  0.6082
⚠️ Hamming Loss:         0.0307
✅ Subset Accuracy:      0.7660
